In [1]:
import numpy as np
import pandas as pd
import pickle
import torch.nn as nn
from torch.utils.data import DataLoader
import json
import plotly.express as px

#Implement training process
from model_trees_algebra import NeoRegression

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

class BaoTrainingException(Exception):
    pass

URL = "/media/data/ccarmona/memoria/dataset/"
csv_name = 'new_dataset_5.4_full'
salt_in_name = csv_name + '-prueba-'
x = [True,False]
active_delete = x[1]
active_new_data = x[0]

IS CUDA AVAILABLE: True


In [2]:
def delete_potential_problems_indexes(df_raw, active=True):
    indexes_columns = []
    c = 0
    if active:
        for i in range(len(df_raw)):
            if 'OPTIONAL' in df_raw['query'][i]:
                indexes_columns.append(i)
                c += 1
            if 'UNION' in df_raw['query'][i]:
                indexes_columns.append(i)
                c += 1
            if df_raw['query'][i].count('SELECT') > 2:
                indexes_columns.append(i)
                c += 1
        return indexes_columns, c
    else:
        return indexes_columns, c

In [3]:
import math
from sklearn.utils import shuffle

df_raw = pd.read_csv(URL + csv_name + ".csv", engine='python', encoding='utf-8')

In [4]:
print(df_raw.columns)
if 'sparql_file' in df_raw.columns:
    df_raw = df_raw.rename(columns = {'sparql_file': 'query'}, inplace = False)


print(df_raw['cpu_p'][0])
print(df_raw['same_seg_p'][0])
print(df_raw['same_page_p'][0])
print(df_raw['wait'][0])
print(df_raw['comp_read_p'][0])
print(df_raw['comp_clw'][0])
print(df_raw['comp_msec'][0])
print(df_raw['query'][0])
print('df_raw.shape',df_raw.shape)
print(df_raw.columns)

df_raw['cpu_p'] = df_raw['cpu_p'].apply(lambda x: float(x.strip('%')))
df_raw['same_seg_p'] = df_raw['same_seg_p'].apply(lambda x: float(x.strip('%')))
df_raw['same_page_p'] = df_raw['same_page_p'].apply(lambda x: float(x.strip('%')))
df_raw['wait'] = df_raw['wait'].apply(lambda x: float(x.strip('%')))
df_raw['comp_read_p'] = df_raw['comp_read_p'].apply(lambda x: float(x.strip('%')))
df_raw['comp_clw'] = df_raw['comp_clw'].apply(lambda x: float(x.strip('%')))


print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("With deletes")
indexes_columns, c = delete_potential_problems_indexes(df_raw, active_delete)
df_raw = df_raw.drop(indexes_columns)
print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("Numero de filas borradas:",c)
print(df_raw['cpu_p'][0])
print(df_raw['same_seg_p'][0])
print(df_raw['same_page_p'][0])
print(df_raw['wait'][0])
print(df_raw['comp_read_p'][0])
print(df_raw['comp_clw'][0])
print(df_raw['comp_msec'][0])
print(df_raw['query'][0])

print('df_raw.shape',df_raw.shape)

print(f'shape df_raw: {df_raw.shape}')

Index(['unique_id', 'filename', 'sparql_file', 'profile', 'limit', 'group_by',
       'distinct', 'order_by', 'union', 'left_join', 'join', 'iter', 'filter',
       'num_filter', 'filter_eq', 'filter_gt', 'filter_ge', 'filter_lt',
       'filter_le', 'filter_neq', 'filter_iri', 'filter_neq.1', 'filter_bound',
       'filter_contains', 'filter_exists', 'filter_isBlank', 'filter_isIRI',
       'filter_isLiteral', 'filter_lang', 'filter_langMatches', 'filter_not',
       'filter_notexists', 'filter_regex', 'filter_sameTerm', 'filter_str',
       'filter_strstarts', 'filter_or', 'filter_and', 'time', 'cpu_p', 'rnd',
       'seq', 'same_seg_p', 'same_page_p', 'disk_reads', 'read_ahead', 'wait',
       'comp_msec', 'comp_reads', 'comp_read_p', 'comp_messages', 'comp_clw',
       'triples', 'total_bgps', 'treesize', 'matrix_format', 'trees',
       'trees_old_format', 'json_time_predicate', 'json_fanout_predicate',
       'json_input_rows_predicate', 'json_cardinality_fanout',
       'json_ca

In [5]:
print("mean", df_raw['time'].mean())
print("std", df_raw['time'].std())

mean 3578.6367573751736
std 136090.7424811686


In [6]:
print('df_raw.shape',df_raw.shape)

df_raw.shape (24474, 65)


In [7]:
df_raw['time'].std()

136090.7424811686

In [8]:
df_raw = df_raw[df_raw['time'] <=100000]

In [9]:
if active_new_data:
    df_raw = shuffle(df_raw)
    total_size=len(df_raw)
    train_size=math.floor(0.85*total_size)
    train=df_raw.head(train_size)
    test=df_raw.tail(len(df_raw) -train_size)
    
    train.to_csv(URL + salt_in_name + '_train_val.csv')
    test.to_csv(URL + salt_in_name +'_test.csv')
    print("New csv generates")
else:
    print("Used old csv")

New csv generates
